In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [11]:
TRAINING_DIR = '/Users/mprad/Med_Img/train'
TEST_DIR='/Users/mprad/Med_Img/test'
VAL_DIR='/Users/mprad/Med_Img/val'

In [12]:
num_classes = 3
IMG_SIZE = 224
BATCH_SIZE = 64
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10


In [13]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    subset='training'
)

Found 9674 images belonging to 3 classes.


In [14]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2)

# Training Gen
val_gen = train_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='categorical',
    subset='training'
)

Found 1208 images belonging to 3 classes.


In [15]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = train_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=True,
        class_mode='categorical',
        subset='validation'
)

Found 301 images belonging to 3 classes.


In [16]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.DenseNet121(
    weights = 'imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

# se acopla el modelo
model = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

model.summary()

____________
conv5_block5_0_relu (Activation (None, 7, 7, 640)    0           conv5_block5_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block5_1_conv (Conv2D)    (None, 7, 7, 128)    81920       conv5_block5_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block5_1_bn (BatchNormali (None, 7, 7, 128)    512         conv5_block5_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block5_1_relu (Activation (None, 7, 7, 128)    0           conv5_block5_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_block5_2_conv (Conv2D)    (None, 7, 7, 32)     36864       conv5_block5_1_relu[0][0]        
________________________________________________________________________________________________

In [18]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = num_epochs)

Epoch 1/10
151/151 [==============================] - 692s 5s/step - loss: 0.9727 - accuracy: 0.8001 - val_loss: 0.4645 - val_accuracy: 0.8602
Epoch 2/10
151/151 [==============================] - 682s 5s/step - loss: 0.3813 - accuracy: 0.8649 - val_loss: 0.3286 - val_accuracy: 0.8872
Epoch 3/10
151/151 [==============================] - 677s 4s/step - loss: 0.3207 - accuracy: 0.8820 - val_loss: 0.3449 - val_accuracy: 0.8898
Epoch 4/10
151/151 [==============================] - 676s 4s/step - loss: 0.3232 - accuracy: 0.8831 - val_loss: 0.2662 - val_accuracy: 0.9106
Epoch 5/10
151/151 [==============================] - 677s 4s/step - loss: 0.2926 - accuracy: 0.8984 - val_loss: 0.3034 - val_accuracy: 0.8828
Epoch 6/10
151/151 [==============================] - 688s 5s/step - loss: 0.2967 - accuracy: 0.8880 - val_loss: 0.2830 - val_accuracy: 0.8967
Epoch 7/10
151/151 [==============================] - 710s 5s/step - loss: 0.3094 - accuracy: 0.8922 - val_loss: 0.3331 - val_accuracy: 0.8898